# AI4Omics Practical Session

## Task 1 -  Introduction to classification models in machine learning

## 1. Import data

Import `pandas` package which allows us to perform data analysis and manipulation of dataframes in Python.

In [ ]:
import pandas as pd

The data are available in the file `colon_cancer.csv`. Data import is done with the pandas `read_csv` command. The `shape` attribute contains the dimensions of the dataframe.

In [ ]:
data = pd.read_csv('../data/colon_cancer.csv', sep=';', index_col='id_sample')
print('data', data.shape)

The `head()` method displays by default the first 5 lines of the dataframe. If necessary, we can indicate the number of lines to display, for example `head(3)` for 3 lines.

In [ ]:
data.head()

The data contain the expression levels of **60 genes** in **804 samples** of colon tissue. The last column `tissue_status` indicates if the sample is normal or tumoral.

The data types of each column can be displayed with `dtypes`.

In [ ]:
data.dtypes

We can see that the data are mostly *float values* except for `tissue_status` which is actually a *string*, considered as an *object* by pandas.

The `describe` method displays descriptive statistics of numerical data only. The `tissue_status` column will not be included.

In [ ]:
data.describe()

## 2. Display distributions of expression levels

To create graphical plots we will use the graphical packages of Python `matplotlib` and `seaborn`.

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

The `displot` command of the `seaborn` package enables us to display a distribution of values (as a kernel density estimator *kde* or as a histogram *hist*). Let's select one gene, for example *DAO*, and plot the distribution of expression levels of this gene across all samples. 

In [ ]:
selected_gene = 'DAO'
sns.displot(data=data, x=selected_gene, hue='tissue_status', kind='kde')

The gene *DAO* is highly expressed in normal samples while its expression in tumour samples is much lower.

We can intuitively feel that if we introduce a certain threshold for *DAO* expression, for example 7.5, we will be able to predict for any new sample if it is normal or tumoral. For this, we just need to measure the level of *DAO* expression in the new sample. If it is below 7.5 than the sample is tumoral, otherwise it is normal.   

## 3. Create features and targets for machine learning 

For machine learning purposes in Python, we usually prepare data in two separate objects:
- The first object is a matrix (or a dataframe) of *data*, typically named  **X**, which contains the measurements for all available variables (features). In our case, the features are the 60 genes. The dataframe **X** contains the expression levels of these genes (features).
- The second object is a list of *targets* that we aim to predict, named **y**. In our example, it corresponds to the column `tissue_status` containing the types of each sample, normal tissue or cancer.

Let's consider a simple case when we have only one gene *DAO*. What would be the data **X** and the targets **y**?  

In this case, we have only one feature, the gene *DAO*. The dataframe **X** will therefore contain the expression levels of this gene in all the samples.

In [ ]:
features = ['DAO']
X = data[features] # dataframe (pandas)
X.head(3)

The targets **y** correspond to the column `tissue_status` containing the types of each sample. It can be implemented as a *list* in Python or a *numpy.array* or a *pandas.Series*. We will use the last option.

In [ ]:
y = data['tissue_status'] # series (pandas)
print(y)

## 4. Create training and test datasets

In a machine learning approach, it is mandatory to split the initial dataset **X** into two datatsets: one dataset **X_train** will be used to train the model and the other **X_test** to test its efficiency. The samples for these datasets will be selected randomly.

To create the datasets **X_train** and **X_test**, we will use the framework `scikit-learn` which contains a dedicated tool `train_test_split`.

In [ ]:
from sklearn.model_selection import train_test_split

Let's use 2/3 of samples from the original dataset **X** as a training set **X_train**, and another 1/3 of samples as a test set **X_test**.

<img src="train_test_split.png" alt="Splitting original dataset in training and test" width="400" aling="center">

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=42, stratify=y)
print('Train dataset:', X_train.shape, 'Test dataset:', X_test.shape)

We automatically generated **X_train** and **X_test** datasets with their corresponding targets **y_train** and **y_test**. 

The option `random_state` in `train_test_split` initializes a random generator. The option `stratify` indicates that the proportions of tumour and normal samples in both **X_train** and **X_test** datasets should be the same as in the initial dataset **X** (50/50 in our case). 

Now, if we display the samples included in the training dataset **X_train**, we can see that the initial samples have been shuffled and randomly selected. The order of samples from the original dataset **X** is not conserved.

In [ ]:
X_train.head(3)

Same for **X_test**.

In [ ]:
X_test.head(3)

## 5. Training a Decision Tree

A decision tree during the training step search for an optimal threshold that allows to separate normal and tumour samples. The `max_depth` option indicates the maximum depth of the tree. The `fit` method performs the training of the model. The training of the model is done only on the training dataset **X_train**.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(max_depth=1, random_state=42, criterion='entropy')
classifier.fit(X_train, y_train)

It could be interesting to display the optimal threshold found by the algorithm during the learning process. We can do it with the `export_text` function.

In [ ]:
from sklearn.tree import export_text, plot_tree
decision_tree_text = export_text(classifier, feature_names=list(X_train.columns))
print(decision_tree_text)

We can also visualize the obtained decision tree when it is not too complex. The `plot_tree` function generates the graph.

In [ ]:
plot_tree(classifier, feature_names=list(X_train.columns),  class_names=y.unique(), precision=2, filled=True)

## 6. Predict the diagnosis of new patients (test dataset) 

Once the model has been trained, it can be used on new samples to predict their status (normal or tumour). The performance of the model is usually estimated by a metric. In our example, we will use the *accuracy* metric.

**Accuracy** = Number of correct predictions / Total number of predictions

The prediction can be done with the `predict` method. In `scikit-learn`,  all the supervised machine learning models have at leat two standard methods: `fit` to train the model (on train dataset) and `predict` to perform a prediction (on test dataset).

In [ ]:
y_pred_train = classifier.predict(X_train)
y_pred_test = classifier.predict(X_test)

The accuracy calculation is available in `metrics` module of `scikit-learn`.

In [ ]:
from sklearn import metrics
accuracy_train = metrics.accuracy_score(y_train, y_pred_train)
accuracy_test = metrics.accuracy_score(y_test, y_pred_test)
print('Train accuracy:', '{:.2f}'.format(accuracy_train), 'Test accuracy:', '{:.2f}'.format(accuracy_test))

To **evaluate a model**, we essentially take into account the **metric calculated on the test dataset**. Sometimes, we can also compare it with the metric obtained on the training dataset to know if the model tends to overfit.

## 7. Create a function that performs all steps

Subsequently, in the following exercises we will often perform the training, prediction and metric calculation steps for different machine learning models. It will be more convenient for us to create a special `calculate_accuracy` function that performs all these steps automatically.

In [ ]:
def calculate_accuracy(classifier, X_train, X_test, y_train, y_test):
    classifier.fit(X_train, y_train)
    y_pred_train = classifier.predict(X_train)
    y_pred_test = classifier.predict(X_test)
    accuracy_train = metrics.accuracy_score(y_train, y_pred_train)
    accuracy_test = metrics.accuracy_score(y_test, y_pred_test)
    print('Train accuracy:', '{:.2f}'.format(accuracy_train), 'Test accuracy:', '{:.2f}'.format(accuracy_test))
    return accuracy_train, accuracy_test, classifier

Example of use: 

In [ ]:
accuracy_train, accuracy_test, trained_classifier = calculate_accuracy(classifier, X_train, X_test, y_train, y_test)

*Disclaimer! The function* `calculate_accuracy` *is proposed here to simplify the code of the lesson, for teaching purposes only. In this example, it is convenient for us to add all the steps in the same function. Nevertheless, if you need to produce a professional code for production purposes, please take into account good practices of software engineering. Usually, the good practices recommend to separate different actions in different atomic functions and don't mix the calculation and the presentation of data/results. A concrete approach may depend on the programming paradigm.*

**Congratulations, you successfully completed the task 1! Please, proceed to the task 2.**